In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
path_files = r'/content/gdrive/MyDrive/Uni/ML-Project/Data/'

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
d1='mfcc/mfccaudio_mfcc_features_40_all_20Sec_cleaned.pkl'
l1='mfcc/mfccaudio_mfcc_features_40_all_20Sec_labels_cleaned.pkl'

d2='mfcc/audio_mfcc_features_40_all_20Sec.pkl'
l2='mfcc/audio_mfcc_features_40_all_20Sec_labels.pkl'

d3='mfcc/mfccaudio_mfcc_features_20_all_20Sec_cleaned_Amin.pkl'
l3='mfcc/mfccaudio_mfcc_features_20_all_20Sec_labels_cleaned_Amin.pkl'

d4='chroma/mfccaudio_chroma_features_12_all_20Sec_cleaned.pkl'
l4='chroma/mfccaudio_chroma_features_12_all_20Sec_labels_cleaned.pkl'

d5='chroma/mfccaudio_chroma_features_24_all_20Sec_cleaned.pkl'
l5='chroma/mfccaudio_chroma_features_24_all_20Sec_labels_cleaned.pkl'


# Data extraction
def data_extract(d):
    X=d.copy()
    for i in range(len(X)):
      f1 = np.mean(X[i],axis=0)
      f2 = np.var(X[i],axis=0)
      X[i] = np.append(f1,f2)
    return X

def file2data(d,l):
  path_data = path_files+d
  path_label = path_files+l
  data = pd.read_pickle(path_data)
  y = pd.read_pickle(path_label)
  X = data_extract(data)
  return X,y


In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

# KNN
def knn_hypertunning(X, y,k_range = [1,10,50,100,150,200,250,300]):
    params = {'n_neighbors':k_range}
    search = GridSearchCV(KNeighborsClassifier(),params,cv=4,n_jobs=-1).fit(X,y)
    best_k = search.best_params_['n_neighbors']
    return best_k

def print_scores(clf, X, y):
    cv_results = cross_validate(clf, X, y, cv=4, 
                                scoring='accuracy',
                                return_train_score='true',
                                n_jobs=-1)
    
    mean_train = np.array(cv_results['train_score']).mean().round(3)
    test = np.array(cv_results['test_score'])
    mean_test = test.mean().round(3)
    sd_test = np.sqrt(test.var()).round(4)
    print(f'accuracy score on test data: ',mean_test,"\u00B1",sd_test)
    print(f'accuracy score on train data:',mean_train)
    return

for d,l in zip((d1,d2,d3,d4,d5),(l1,l2,l3,l4,l5)):
  X,y = file2data(d,l)
  lda = LinearDiscriminantAnalysis().fit(X,y)
  X_lda = lda.transform(X)
  k = knn_hypertunning(X_lda,y)
  knn = KNeighborsClassifier(n_neighbors=k)
  print('\nfile name:',d)
  print('k=',k)
  print_scores(knn, X_lda, y)


file name: mfcc/mfccaudio_mfcc_features_40_all_20Sec_cleaned.pkl
k= 200
accuracy score on test data:  0.404 ± 0.0402
accuracy score on train data: 0.468

file name: mfcc/audio_mfcc_features_40_all_20Sec.pkl
k= 300
accuracy score on test data:  0.399 ± 0.038
accuracy score on train data: 0.459

file name: mfcc/mfccaudio_mfcc_features_20_all_20Sec_cleaned_Amin.pkl
k= 300
accuracy score on test data:  0.269 ± 0.0381
accuracy score on train data: 0.378

file name: chroma/mfccaudio_chroma_features_12_all_20Sec_cleaned.pkl
k= 200
accuracy score on test data:  0.325 ± 0.029
accuracy score on train data: 0.408

file name: chroma/mfccaudio_chroma_features_24_all_20Sec_cleaned.pkl
k= 300
accuracy score on test data:  0.367 ± 0.0468
accuracy score on train data: 0.425
